# Proyecto individual ML_Ops Movies

In [54]:
import pandas as pd
from pandas import json_normalize
import numpy as np
from datetime import datetime
import ast  # Importar el módulo ast

Hacemos una interseccion con los dos dataframe df_movies y df_credits

In [3]:
df_credits = pd.read_csv("c:\\Users\\Giuliano\\Desktop\\credits.csv")
df_movies = pd.read_csv("c:\\Users\\Giuliano\\Desktop\\movies_dataset.csv",low_memory=False)

In [4]:
# Convertir la columna 'id' en ambos DataFrames a tipo string
df_movies['id'] = df_movies['id'].astype(str)
df_credits['id'] = df_credits['id'].astype(str)

In [5]:
# Merge entre df_movies y df_credits usando la columna 'id' en df_movies y 'id' en df_credits.
df = pd.merge(df_movies, df_credits, left_on='id', right_on='id', how='inner')

ETL

In [6]:
# Cambiar el nombre de la columna 'id' en df a 'movie_id'
df.rename(columns={'id': 'movie_id'}, inplace=True)

Desanidamos belongs_to_collection

In [7]:
# Convertir las cadenas en diccionarios
df['belongs_to_collection'] = df['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else None)


In [8]:
df_desanidado = pd.json_normalize(df['belongs_to_collection'])

In [9]:
# Unir el DataFrame desanidado con el DataFrame original
df= pd.concat([df, df_desanidado], axis=1)

In [10]:
# Cambiar el nombre de la columna 'id' en df_credits a 'movie_id'
df.rename(columns={'id': 'franquicia_id'}, inplace=True)
# Eliminar la columna ''
df.drop(columns=['poster_path'], inplace=True)

Desanidamos product_companies

In [11]:
# Convertir las cadenas en listas de diccionarios, manejando NaN y None
df['production_companies'] = df['production_companies'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else None
)

# Expandir las listas en filas separadas
df_expanded = df.explode('production_companies')
# Convertir los diccionarios en columnas separadas
df_companies = pd.json_normalize(df_expanded['production_companies'])

In [12]:
df = pd.concat([df,df_companies], axis=1)

In [13]:
# Borramos las columnas bellong_to_collection y product_companies
df = df.drop(columns=['belongs_to_collection', 'production_companies'])

In [14]:
df.columns

Index(['adult', 'budget', 'genres', 'homepage', 'movie_id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew', 'franquicia_id', 'name',
       'backdrop_path', 'name', 'id'],
      dtype='object')

In [15]:
# Reemplazar los valores nulos en las columnas 'revenue' y 'budget' por 0
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df_movies['budget'].fillna(0)

In [16]:
# Eliminar las filas donde el valor de 'release_date' es nulo
df = df.dropna(subset=['release_date'])

In [17]:
# Verificar cuántos valores nulos quedan en 'release_date'
nulos_release_date = df['release_date'].isnull().sum()
nulos_release_date

np.int64(0)

In [18]:
# Asegurarse de que 'release_date' esté en formato de fecha AAAA-mm-dd
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d',errors="coerce")

In [19]:
# Eliminar las filas donde 'release_date' no pudo convertirse (valores nulos)
df = df.dropna(subset=['release_date'])

In [20]:
# Crear una nueva columna 'release_year' extrayendo el año de 'release_date'
df['release_year'] = df['release_date'].dt.year

In [21]:
# Asegurarse de que 'revenue' y 'budget' sean numéricos, reemplazar valores no convertibles por 0
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce').fillna(0)
df['budget'] = pd.to_numeric(df['budget'], errors='coerce').fillna(0)

In [40]:
df["retorno_de_inversion"] = df["revenue"] / df["budget"] 

In [51]:
df["retorno_de_inversion"] = df["retorno_de_inversion"].fillna(0)

In [52]:
df["retorno_de_inversion"] = df["retorno_de_inversion"][
        (df['retorno_de_inversion'] != np.inf) &  # Excluir infinito
        (df['retorno_de_inversion'] != -np.inf) &  # Excluir -infinito
        (df['retorno_de_inversion'].notna()) &  # Excluir NaN
        (df['retorno_de_inversion'] >= 0)  # Asegurarse de que el valor sea positivo
    ]

In [53]:
# Si está correcto, elimina las columnas
df = df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'homepage'])

In [69]:
df.columns

Index(['budget', 'genres', 'movie_id', 'original_language', 'overview',
       'popularity', 'production_countries', 'release_date', 'revenue',
       'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'franquicia_id', 'name',
       'backdrop_path', 'name', 'id', 'release_year', 'retorno_de_inversion'],
      dtype='object')

In [70]:
df.to_csv("df.csv")

Desanidamos cast y crew

In [6]:
df_cast = pd.DataFrame(df['cast'])

In [9]:
# Convertir las cadenas en listas de diccionarios, manejando NaN y None
df_cast['cast'] = df_cast['cast'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else None
)

In [10]:
# Expandir las listas en filas separadas
df_cast = df_cast.explode('cast')

In [11]:
# Convertir los diccionarios en columnas separadas
df_cast = pd.json_normalize(df_cast['cast'])

In [12]:
df_cast.rename(columns={'id': 'actor_id'}, inplace=True)

In [13]:
df.columns

Index(['Unnamed: 0', 'budget', 'genres', 'movie_id', 'original_language',
       'overview', 'popularity', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'franquicia_id', 'name',
       'backdrop_path', 'name.1', 'id', 'release_year',
       'retorno_de_inversion'],
      dtype='object')

In [14]:
df_crew = pd.DataFrame(df['crew'])

In [15]:
# Convertir las cadenas en listas de diccionarios, manejando NaN y None
df_crew['crew'] = df_crew['crew'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else None
)

# Expandir las listas en filas separadas
df_crew = df_crew.explode('crew')

# Convertir los diccionarios en columnas separadas
df_crew = pd.json_normalize(df_crew['crew'])


In [18]:
df_crew.to_csv("df_crew.csv")
df_cast.to_csv("df_cast.csv")

In [19]:
df = pd.concat([df,df_crew],axis=1)

In [20]:
df = pd.concat([df,df_cast],axis=1)

In [21]:
df.columns

Index(['Unnamed: 0', 'budget', 'genres', 'movie_id', 'original_language',
       'overview', 'popularity', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'franquicia_id', 'name',
       'backdrop_path', 'name.1', 'id', 'release_year', 'retorno_de_inversion',
       'credit_id', 'department', 'gender', 'id', 'job', 'name',
       'profile_path', 'cast_id', 'character', 'credit_id', 'gender',
       'actor_id', 'name', 'order', 'profile_path'],
      dtype='object')

In [23]:
df = df.drop(columns=['id', 'job', 'department','gender','backdrop_path','cast', 'crew','runtime', 'spoken_languages', 'status', 'tagline', 'production_countries','overview'])

In [25]:
df.columns

Index(['Unnamed: 0', 'budget', 'genres', 'movie_id', 'original_language',
       'popularity', 'release_date', 'revenue', 'title', 'vote_average',
       'vote_count', 'franquicia_id', 'name', 'name.1', 'release_year',
       'retorno_de_inversion', 'credit_id', 'name', 'profile_path', 'cast_id',
       'character', 'credit_id', 'actor_id', 'name', 'order', 'profile_path'],
      dtype='object')

In [26]:
df.to_csv("df_movies.csv")